![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_XLMRoBERTaForMultipleChoice.ipynb)

# Import OpenVINO XlmRoBertaForMultipleChoice models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting XlmRoBertaForMultipleChoice  models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for XlmRoBertaForMultipleChoice from XlmRoBertaForMultipleChoice  and they have to be in `Multiple Choice` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.48.3 optimum-intel openvino

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- Normally, to load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. However, ForMultipleChoice is not yet available so we will use `openvino.convert_model()` after exporting ONNX model
- We'll use [lenatr99/fine_tuned_copa_XLMroberta](https://huggingface.co/lenatr99/fine_tuned_copa_XLMroberta) model from HuggingFace as an example
- In addition to the RoBERTa model, we also need to save the tokenizer. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from optimum.onnxruntime import ORTModelForMultipleChoice

MODEL_NAME = "lenatr99/fine_tuned_copa_XLMroberta"
ONNX_MODEL_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForMultipleChoice.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL_PATH)

Multiple distributions found for package optimum. Picked distribution: optimum-intel
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/714 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Save Tokenizer for OpenVINO

In [10]:
from transformers import AutoTokenizer

OPENVINO_MODEL_PATH = "openvino_models"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(OPENVINO_MODEL_PATH)

('openvino_models/tokenizer_config.json',
 'openvino_models/special_tokens_map.json',
 'openvino_models/sentencepiece.bpe.model',
 'openvino_models/added_tokens.json',
 'openvino_models/tokenizer.json')

Convert ONNX to OpenVINO IR Format

In [4]:
import openvino as ov

ov_model = ov.convert_model(f"{ONNX_MODEL_PATH}/model.onnx")
ov.save_model(ov_model, f"{OPENVINO_MODEL_PATH}/openvino_model.xml")

Organize Assets

In [11]:
!mkdir {OPENVINO_MODEL_PATH}/assets

In [12]:
!mv {OPENVINO_MODEL_PATH}/sentencepiece.bpe.model {OPENVINO_MODEL_PATH}/assets

In [13]:
!ls -l {OPENVINO_MODEL_PATH}/assets

total 4952
-rw-r--r-- 1 root root 5069051 Jul  2 17:33 sentencepiece.bpe.model


## Import and Save XlmRoBertaForMultipleChoice in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [7]:
!pip install -q spark-nlp==6.0.3 pyspark==3.5.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/713.0 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 706.6/713.0 kB 25.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.0/713.0 kB 17.6 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [8]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  6.0.3
Apache Spark version:  3.5.1


- Let's use `loadSavedModel` functon in `XlmRoBertaForMultipleChoice` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `XlmRoBertaForMultipleChoice` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [14]:
from sparknlp.annotator import XlmRoBertaForMultipleChoice

xlmRoBertaMultpleChoiceClassifier = XlmRoBertaForMultipleChoice.loadSavedModel(
     f"{OPENVINO_MODEL_PATH}",
     spark
 )\
  .setInputCols(["document_question", "document_context"])\
  .setOutputCol("answer")

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [15]:
xlmRoBertaMultpleChoiceClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [16]:
!rm -rf {ONNX_MODEL_PATH}
!rm -rf {OPENVINO_MODEL_PATH}

Awesome 😎  !

This is your `XlmRoBertaForMultipleChoice` model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [17]:
! ls -l {MODEL_NAME}_spark_nlp_openvino

total 548692
drwxr-xr-x 2 root root      4096 Jul  2 17:34 metadata
-rw-r--r-- 1 root root 556777491 Jul  2 17:35 xlm_roberta_mc_classification_openvino
-rw-r--r-- 1 root root   5069051 Jul  2 17:34 xlmroberta_spp


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny `XlmRoBertaForMultipleChoice` model 😊

In [18]:
testing_data = spark.createDataFrame([
    ("In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.", "It is eaten with a fork and a knife, It is eaten while held in the hand."),
    ("The Eiffel Tower is located in which country?", "Germany, France, Italy"),
    ("Which animal is known as the king of the jungle?", "Lion, Elephant, Tiger, Leopard"),
    ("Water boils at what temperature?", "90°C, 120°C, 100°C"),
    ("Which planet is known as the Red Planet?", "Jupiter, Mars, Venus"),
    ("Which language is primarily spoken in Brazil?", "Spanish, Portuguese, English"),
    ("The Great Wall of China was built to protect against invasions from which group?", "The Greeks, The Romans, The Mongols, The Persians"),
    ("Which chemical element has the symbol 'O'?", "Oxygenm, Osmium, Ozone"),
    ("Which continent is the Sahara Desert located in?", "Asia, Africa, South America"),
    ("Which artist painted the Mona Lisa?", "Vincent van Gogh, Leonardo da Vinci, Pablo Picasso")
], ["question", "choices"])

testing_data.show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|question                                                                                  |choices                                                                 |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  |
|Which animal is known as the king of the jungle?                                          |Lion, Elephant, Tiger, Leopard                                          |
|Wat

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [21]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import XlmRoBertaForMultipleChoice
from pyspark.ml import Pipeline

assembler = MultiDocumentAssembler()\
    .setInputCols(["question", "choices"])\
    .setOutputCols(["document_question", "document_choices"])

XlmRoBerta_model = XlmRoBertaForMultipleChoice()\
    .load(f"{MODEL_NAME}_spark_nlp_openvino")\
    .setInputCols(["document_question", "document_choices"])\
    .setOutputCol("answer")\
    .setBatchSize(4)

pipeline = Pipeline(stages=[
    assembler,
    XlmRoBerta_model
])

model = pipeline.fit(testing_data)
result = model.transform(testing_data)

result.selectExpr("question", "choices", "answer.result[0] as predicted_answer").show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+-----------------------------------+
|question                                                                                  |choices                                                                 |predicted_answer                   |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+-----------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.|It is eaten with a fork and a knife|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  |Germany                    

That's it! You can now go wild and use hundreds of `XlmRoBertaForMultipleChoice` models from HuggingFace 🤗 in Spark NLP 🚀
